# Auto-Analyser-UAT Stats
### Follow these steps to generate statistics of names tested against the auto analyser

## Setup the environment

In [1]:
%run ./notebook-setup.ipynb

## Set the data set
#### Default: entire data set of results that haven't been seen
*Could be expanded upon to select other subsets of uat jobs / names*

In [2]:
data_set = 'unseen'  # all, unseen
uat_type = 'all'  # uat_rejection, uat_accuracy, all

## Get the uat jobs included in the data set

In [3]:
if data_set == 'unseen':
    if uat_type != 'all':
        uat_jobs = UatJobResult.get_jobs_with_unsent_results(uat_type=uat_type)
    else:
        uat_jobs = UatJobResult.get_jobs_with_unsent_results()
elif uat_type != 'all':
    uat_jobs = UatJobResult.get_finshed_jobs(uat_type=uat_type)
else:
    uat_jobs = UatJobResult.get_finshed_jobs()
uat_jobs

[<UatJobResult 1>,
 <UatJobResult 2>,
 <UatJobResult 3>,
 <UatJobResult 4>,
 <UatJobResult 5>,
 <UatJobResult 85>,
 <UatJobResult 86>,
 <UatJobResult 88>,
 <UatJobResult 37>,
 <UatJobResult 129>,
 <UatJobResult 130>]

## Get the response time averages

In [4]:
total_request_time = 0.0
count = 0
for job in uat_jobs:
    time = job.get_request_time_avg()
    print(time)
    total_request_time += time
    count += 1
total_request_time_avg = total_request_time/count
total_request_time_avg

6.3
6.9
3.4
8.6
6.9
6.5
4.1
6.9
4.8
10.0
7.0


6.49090909090909

## Get the pass rate

In [5]:
total_pass_avgs = 0.0
count = 0
for job in uat_jobs:
    pass_avg = job.get_accuracy()
    print(pass_avg)
    total_pass_avgs += pass_avg
    count += 1
total_pass_rate = total_pass_avgs/count
total_pass_rate

1.0
0.8
0.6
0.4
0.6
0.7
0.6
0.8
0.3
1.0
0.0


0.618181818181818

## Get the failed results

In [6]:
failed_names = []
for job in uat_jobs:
    failed_names += job.get_names(result='FAILED')
for name_obj in failed_names:
    print(name_obj.name)


TRISTON'S DRY CLEANING INC.
SCOTT OWEN RUMSBY INC.


## Save failed results to excel spreadsheet

In [7]:
write_to_excel(failed_names, 'failed_names.xlsx')

## Get err'd results

In [8]:
errd_names = []
for job in uat_jobs:
    errd_names += job.get_names(result='ERROR')
for name_obj in errd_names:
    print(name_obj.name)

BLUE WATER RESORT INC.
BEST VICTORIA PLUMBING INC.
JJ FIT FOR LIFE INCORPORATED
F. GERARD BOOK KEEPING 2 INC.
VICTORIA PLUMBING INC.
WILX & SONS HEAT SAVERS INC.
WILX HEAT & AIR CONDITIONING INC.
COAST NOTARY CORPORATION UNLIMITED LIABILITY COMPANY LIMITED
HH WELDING LIMITED
ROYAL BC GUSTO MARKETING LIMITED
FIRST CHOICE MORTGAGE LIMITED
BC PRESTIGE HOLDINGS LTD. LIMITED
BC BUILDERS ASSOCIATION LIMITED
BACK COUNTRY BUILDERS AND GUIDES LIMITED
ALL CARE TRUCK REPAIR LTD.
I AM ADDING A NEW NAME CHOICE
GREEN LIGHT CONSULTING INC. LIMITED
ECO TRANSPORTATION GROUP LTD. LIMITED
DYNAMIC LEASING CANADA INC. LIMITED
DYNAMIC BUSINESS SOLUTIONS LIMITED
CS PROPERTY SERVICES LTD. LIMITED
B C WIDE TRANSPORT LTD. LIMITED
J AND J PAINTING LIMITED
JJ PAINTING LIMITED
ACORN MANAGEMENT CONSULTANTS, LP LIMITED
ACORN HOLDINGS, LP LIMITED
A&K DELIVERIES LIMITED
A-MATE CONSTRUCTION SERVICES LIMITED
QUANTITY BUILDING MATERIALS
KAHN CLEANING AND LAUNDRY INCORPORATED
KELLIE'S LAUNDRY INC.


In [9]:
write_to_excel(errd_names, 'error_names.xlsx')

## Update jobs results sent

In [10]:
for job in uat_jobs:
    job.results_sent = True
    job.save()
db.session.commit()

## Create MD file of notebook run
**NOTE:** save notebook (i.e. _cmd s_) now to have results show in markdown file

In [11]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [12]:
md_name = nb_name[:-6]+'.md'

In [13]:
%%bash -s "$nb_name" "$md_name"
jupyter nbconvert $1 --to markdown --output $2

[NbConvertApp] Converting notebook auto-analyser-uat-stats.ipynb to markdown
[NbConvertApp] Writing 4093 bytes to auto-analyser-uat-stats.md
